In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Start SparkSession
spark = SparkSession.builder \
    .appName("Online Banking Analysis") \
    .getOrCreate()
# Load loan.csv
loan_df = spark.read.csv("/content/loan.csv", header=True, inferSchema=True)

# Load credit card.csv
credit_df = spark.read.csv("/content/credit card.csv", header=True, inferSchema=True)

# Load txn.csv
txn_df = spark.read.csv("/content/txn.csv", header=True, inferSchema=True)

# Display schema
loan_df.printSchema()
credit_df.printSchema()
txn_df.printSchema()

# Show first few records
loan_df.show(5)
credit_df.show(5)
txn_df.show(5)


root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: dou

In [3]:
loan_df.groupBy("Loan Category").count().show()


+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [5]:
for col_name in loan_df.columns:
    loan_df = loan_df.withColumnRenamed(col_name, col_name.strip().replace(" ", "_").replace("-", "_"))


In [6]:

# Filtering customers who took a loan greater than 1 lakh
loan_df.filter(loan_df["Loan_Amount"] > 100000).count()

0

In [7]:

# Filtering rows where income is greater than 60,000
loan_df.filter(col("Income") > 60000).count()


198

In [10]:
# Applying multiple conditions using logical AND (&)
loan_df.filter((col("Returned_Cheque") >= 2) & (col("Income") < 50000)).count()

137

In [11]:
# Filtering by returned cheques and marital status
loan_df.filter((col("Returned_Cheque") >= 2) & (col("Marital_Status") == "Single")).count()

0

In [12]:
# Filtering high monthly spenders
loan_df.filter(col("Expenditure") > 50000).count()

6

In [13]:
# Example rule: Eligible if debt record is good and they use banking services frequently
loan_df.filter((col("Use_Frequency") > 10) & (col("Debt_Record") == "Good")).count()

0

In [14]:
for col_name in credit_df.columns:
    credit_df = credit_df.withColumnRenamed(col_name, col_name.strip().replace(" ", "_").replace("-", "_"))

In [15]:
for col_name in txn_df.columns:
    txn_df = txn_df.withColumnRenamed(col_name, col_name.strip().replace(" ", "_").replace("-", "_"))

In [16]:
# 1. Show number of credit card users located in Spain

credit_df.filter(col("Geography") == "Spain").count()


2477

In [17]:
# Example threshold: Credit Score >= 650 is eligible
credit_df.filter((col("CreditScore") >= 650) & (col("IsActiveMember") == 1)).count()

2672

In [18]:
credit_df.filter((col("CreditScore") >= 650) & (col("IsActiveMember") == 1)).show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        7|  15592531| Bartlett|        822|   France|  Male| 50|     7|      0.0|            2|             1|        10062.8|     0|
|       10|  15592389|       H?|        684|   France|  Male| 27|     2|134603.88|            1|             1|       71725.73|     0|
|       20|  15568982|      Hao|        726|   France|Female| 24|     6|      0.0|            2|             1|       54724.03|     0|
|       21|  15577657| McDonald|        732|   France| 

In [25]:
from pyspark.sql.functions import max, col

txn_df.select(
    max(col("WITHDRAWAL_AMT").cast("double")).alias("Max_Withdrawal")
).show()


+--------------+
|Max_Withdrawal|
+--------------+
|         4.0E8|
+--------------+



In [23]:
#Minimum Withdrawal Amount
txn_df.select(min("WITHDRAWAL_AMT").alias("Min_Withdrawal_Amount")).show()

+---------------------+
|Min_Withdrawal_Amount|
+---------------------+
|                 0.01|
+---------------------+



In [26]:
#Maximum Deposit Amount
txn_df.select(max("DEPOSIT_AMT").alias("Max_Deposit_Amount")).show()

+------------------+
|Max_Deposit_Amount|
+------------------+
|           9999999|
+------------------+



In [28]:
# Minimum Deposit Amount
txn_df.select(min("DEPOSIT_AMT").alias("Min_Deposit_Amount")).show()


+------------------+
|Min_Deposit_Amount|
+------------------+
|       -1667026170|
+------------------+



In [29]:

#Sum of Balance Amount in Each Bank Account

txn_df.groupBy("Account_No") \
      .sum("BALANCE_AMT") \
      .withColumnRenamed("sum(BALANCE_AMT)", "Total_Balance") \
      .show()


+-------------+--------------------+
|   Account_No|       Total_Balance|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.37810146616539...|
|     1196428'| -2.8115133425562E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12237285703308...|
+-------------+--------------------+



In [30]:
# Number of Transactions Happening on Each Date

txn_df.groupBy("VALUE_DATE") \
      .agg(count("*").alias("Transaction_Count")) \
      .orderBy("VALUE_DATE") \
      .show()

+----------+-----------------+
|VALUE_DATE|Transaction_Count|
+----------+-----------------+
|      NULL|                1|
|  1-Aug-15|               27|
|  1-Aug-16|               46|
|  1-Aug-17|               13|
|  1-Aug-18|               65|
|  1-Dec-16|               73|
|  1-Dec-17|                2|
|  1-Dec-18|               38|
|  1-Feb-16|               51|
|  1-Feb-17|               44|
|  1-Feb-18|               17|
|  1-Feb-19|               37|
|  1-Jan-15|                1|
|  1-Jan-18|               22|
|  1-Jan-19|               33|
|  1-Jul-15|                4|
|  1-Jul-16|               65|
|  1-Jul-17|              191|
|  1-Jun-16|               72|
|  1-Jun-17|               14|
+----------+-----------------+
only showing top 20 rows



In [31]:
# Customers With Withdrawal Amount Greater Than ₹1,00,000

txn_df.filter(col("WITHDRAWAL_AMT") > 100000) \
      .select("Account_No", "WITHDRAWAL_AMT", "VALUE_DATE") \
      .orderBy(col("WITHDRAWAL_AMT").desc()) \
      .show()


+-------------+--------------+----------+
|   Account_No|WITHDRAWAL_AMT|VALUE_DATE|
+-------------+--------------+----------+
|409000438620'|       9947700| 15-May-17|
|409000438620'|        994755| 30-Jan-17|
|409000438620'|       9939786| 23-May-17|
|409000438620'|        993499| 14-Mar-17|
|409000438611'|       9900000| 15-Jan-18|
|409000438611'|       9900000| 31-Jul-18|
|409000438620'|       9900000|  2-May-17|
|409000438620'|       9900000|  2-May-18|
|409000438620'|        983537| 11-Jan-17|
|409000438611'|       9800000| 14-Dec-18|
|409000438620'|        977095| 27-Jan-17|
|409000438620'|       9710101| 17-Jan-19|
|409000438611'|       9700000| 27-Aug-18|
|409000438611'|       9700000| 25-Sep-18|
|409000438611'|       9700000| 27-Nov-18|
|409000438620'|       9700000| 26-Oct-17|
|409000438620'|       9600000| 16-May-17|
|409000438620'|       9600000| 30-Nov-17|
|409000438620'|       9600000| 23-Feb-18|
|409000438620'|       9600000|  7-Mar-18|
+-------------+--------------+----